In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4

In [2]:
from typhon.retrieval.qrnn import set_backend, QRNN
set_backend("pytorch")

quantiles = np.array([0.05, 0.15, 0.25, 0.35, 0.45, 0.5, 0.55, 0.65, 0.75, 0.85, 0.95])
#quantiles = np.array([0.05, 0.32, 0.5, 0.68,  0.95])
qrnn = QRNN(4, quantiles, (3, 256 , "relu"))
batchSize = 256

In [3]:
from torch.utils.data import DataLoader, random_split
from aws import awsData
inChannels = ['C34', 'C41', 'C42', 'C43']
train_data = awsData("/home/inderpreet/Dendrite/Projects/AWS-325GHz/TB_AWS/training_data.nc", inChannels, 
                 batch_size = batchSize)

training_data = DataLoader(train_data, batch_size = None, shuffle = False)

val_data = awsData("/home/inderpreet/Dendrite/Projects/AWS-325GHz/TB_AWS/validation_data.nc", inChannels, 
                 batch_size = batchSize)

validation_data = DataLoader(val_data, batch_size = None, shuffle = False)


In [4]:
#for i, batch in enumerate(validation_data):
#        print(i, batch)
#qrnn.train?

In [ ]:
qrnn.train(
    training_data,
    validation_data,
    batch_size=batchSize,
    momentum = 0,
    sigma_noise=None,
    initial_learning_rate=0.001,
    maximum_epochs=40,
    gpu=True)


In [ ]:
#qrnn.calibration?
qrnn.save("qrnn.nc")

In [ ]:
x, y = qrnn.calibration(validation_data)
f, ax = plt.subplots(1, 1)
ax.plot(x, y, marker = "o", c = 'r')
ax.plot(x, x, ls = ":", c = "k")
#ax.set_xlim([0.1, 0.9])
#ax.set_ylim([0.1, 0.9])
ax.set_aspect(1.0)
ax.set_xlabel("Predicted frequency")
ax.set_ylabel("Observed frequency")


In [ ]:
y_pre = []
y = []
y_prior = []

nbatch = val_data.__len__()
print (nbatch)
for i in range(nbatch):
    
    xx, yy = val_data.__getitem__(i)
    
    x = xx.detach().numpy() 

    y_prior.append((x[:, 0] * val_data.std[0]) + val_data.mean[0])
#    y_prior.append((x[:, 0]))

    y_pre.append(qrnn.predict(x).detach().numpy()) 
       
    y.append(yy.detach().numpy())
  

In [ ]:
y_pre = np.concatenate(y_pre, axis = 0)
y = np.concatenate(y, axis= 0)
x = np.concatenate(y_prior, axis = 0)
x.shape

In [ ]:
y_pre[:, 5]

In [ ]:
dtb_pre = y_pre[:, 5] - y
dtb_prior = x - y
binstep = 0.2
bins = np.arange(-40, 20, binstep)
hist_pre = np.histogram(dtb_pre, bins, density = True)
hist_prior = np.histogram(dtb_prior, bins, density = True)

fig, ax = plt.subplots()
ax.plot(bins[:-1], hist_pre[0])
ax.plot(bins[:-1], hist_prior[0])
ax.set_yscale('log')

In [ ]:
x

In [ ]:
import xarray
import os
tb = xarray.open_dataset(os.path.expanduser('~/Dendrite/Projects/AWS-325GHz/TB_AWS/validation_data.nc'))
TB = tb.TB_noise
y_als = np.float32(TB[4, 1, :])
y_cls = np.float32(TB[4, 0, :])


y_pre = qrnn.predict()

In [ ]:
scores = qrnn.crps(y_pre, y, quantiles)
np.mean(scores)

In [ ]:
xs = []
fs = []
for i in range(xx.shape[0]):
    x, f = qrnn.pdf(xx[i])
    

    

In [ ]:
f
